In [1]:
# --- Bloco 0: Importações ---
import gurobipy as gp
from gurobipy import GRB
import time
import numpy as np
import joblib       # Para carregar os scalers
import torch
import torch.nn as nn

In [2]:
import torch
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, n_layers, n_neurons):
        super(Net, self).__init__()
        
        layers = []
        
        # Camada de entrada
        layers.append(nn.Linear(1, n_neurons))
        layers.append(nn.ReLU())
        
        # Camadas ocultas dinâmicas
        for _ in range(n_layers - 1):
            layers.append(nn.Linear(n_neurons, n_neurons))
            layers.append(nn.ReLU())
            
        # Camada de saída
        layers.append(nn.Linear(n_neurons, 1))
        
        # ✅ Adiciona ReLU como ativação final
        layers.append(nn.ReLU())
        
        # Compila todas as camadas em um modelo sequencial
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

print("Definição da classe 'Net' concluída.")


Definição da classe 'Net' concluída.


In [3]:
# --- Bloco 2: Carregar Modelo, Scalers e Extrair Pesos ---

print("--- Carregando Artefatos de Treinamento ---")
# 1. Carregar os Scalers
try:
    x_scaler = joblib.load('x_scaler.joblib')
    y_scaler = joblib.load('y_scaler.joblib')
except FileNotFoundError:
    print("ERRO: Arquivos 'x_scaler.joblib' ou 'y_scaler.joblib' não encontrados.")
    print("Por favor, rode o notebook 'treinamento_redes.ipynb' primeiro.")
    # (Parar a execução do notebook aqui)

# 2. Re-instanciar a arquitetura EXATA que foi salva
# (Do nosso notebook de transfer-learning)
#'''
ARCH_N_LAYERS = 2
ARCH_N_NEURONS = 48
MODEL_PATH = 'temp.pth'
#'''
'''
ARCH_N_LAYERS = 1
ARCH_N_NEURONS = 37
MODEL_PATH = 'zzzzz_final.pth'
#'''
'''
ARCH_N_LAYERS = 2
ARCH_N_NEURONS = 13
MODEL_PATH = 'transfer_model_final_limited.pth'
#'''
'''
ARCH_N_LAYERS = 3
ARCH_N_NEURONS = 20
MODEL_PATH = 'transfer_model_final_best.pth'
#'''
# 3. Instanciar o modelo e carregar os pesos
trained_model = Net(n_layers=ARCH_N_LAYERS, n_neurons=ARCH_N_NEURONS)
try:
    trained_model.load_state_dict(torch.load(MODEL_PATH))
except FileNotFoundError:
    print(f"ERRO: Modelo '{MODEL_PATH}' não encontrado.")
    # (Parar a execução do notebook aqui)
    
trained_model.eval() # Colocar em modo de avaliação
print(f"Modelo '{MODEL_PATH}' carregado.")

# 4. Extrair Pesos (Weights) e Vieses (Biases)
# Vamos salvá-los em um formato que o Gurobi possa ler (NumPy)
nn_params = {}
layer_count = 0
for name, param in trained_model.named_parameters():
    if 'weight' in name:
        nn_params[f'W{layer_count}'] = param.data.cpu().numpy()
    elif 'bias' in name:
        nn_params[f'b{layer_count}'] = param.data.cpu().numpy()
        layer_count += 1

print(f"Pesos e vieses de {layer_count} camadas extraídos.")
# nn_params agora se parece com:
# {'W0': array, 'b0': array, 'W1': array, 'b1': array, 'W2': array, 'b2': array}

--- Carregando Artefatos de Treinamento ---
Modelo 'temp.pth' carregado.
Pesos e vieses de 3 camadas extraídos.


In [4]:
# --- Bloco 3: Função de Cálculo de Bounds (Bound Tightening) ---
#
# Esta é a parte mais crítica da formulação MILP-ReLU.
# Precisamos saber os limites (Mínimo L, Máximo U) da *entrada*
# de cada neurônio (W*x + b) para criar as restrições "Big-M".
#
# Usaremos "Aritmética Intervalar".

def calculate_layer_bounds(input_bounds_list, W, b):
    """
    Calcula os bounds de pré-ativação (Wx+b) para uma camada.
    
    :param input_bounds_list: Lista de tuplas [(L1, U1), (L2, U2), ...] 
                              dos *inputs* desta camada.
    :param W: Matriz de pesos (NumPy) da camada.
    :param b: Vetor de vieses (NumPy) da camada.
    :return: Lista de tuplas [(L_out1, U_out1), ...] dos bounds de 
             *pré-ativação* (antes do ReLU).
    """
    
    n_neurons_out = W.shape[0]
    n_neurons_in = W.shape[1]
    
    # Criar vetores L e U dos inputs
    L_in = np.array([b[0] for b in input_bounds_list])
    U_in = np.array([b[1] for b in input_bounds_list])

    # Separar pesos em positivos e negativos
    W_pos = np.maximum(W, 0)
    W_neg = np.minimum(W, 0)

    # Calcular L e U para cada neurônio de saída
    # L_out = W_pos * L_in + W_neg * U_in + b
    # U_out = W_pos * U_in + W_neg * L_in + b
    
    L_out = W_pos @ L_in + W_neg @ U_in + b
    U_out = W_pos @ U_in + W_neg @ L_in + b

    return list(zip(L_out, U_out))

print("Função 'calculate_layer_bounds' definida.")

Função 'calculate_layer_bounds' definida.


In [5]:
# --- Bloco 4: Pré-Cálculo de TODOS os Bounds da Rede ---

print("--- Calculando Bounds (Big-M) para todos os neurônios ---")

# 1. Definir os bounds de entrada da rede
# A rede foi treinada no domínio [30, 200]
# O scaler transforma [30, 200] para [0, 1]
# O input para o *modelo Gurobi* P_carvao,t é [0, 200]
# (P=0 quando z=0)

# O que acontece com P=0 (quando z=0)?
# P_scaled = (0 - x_scaler.min_[0]) * x_scaler.scale_[0]
# P_scaled = (0 - 30) / (200 - 30) * 1 = -30 / 170 = -0.176
#
# O que acontece com P=30 (mínimo, z=1)?
# P_scaled = (30 - 30) * ... = 0
#
# O que acontece com P=200 (máximo, z=1)?
# P_scaled = (200 - 30) * ... = 1

# Portanto, o input *escalado* que a rede verá está na faixa [-0.176, 1.0]
input_L = (0 - x_scaler.min_[0]) * x_scaler.scale_[0]
input_U = (200 - x_scaler.min_[0]) * x_scaler.scale_[0]

# Bounds da Camada 0 (Input da Rede)
# Lista de tuplas (L, U)
current_bounds = [(input_L, input_U)] 
print(f"Bounds do Input Escalado da Rede (L0): [{input_L:.3f}, {input_U:.3f}]")

# Guardar os bounds de todas as camadas
all_bounds = {}

# 2. Calcular bounds para as camadas ReLU
n_relu_layers = ARCH_N_LAYERS
for i in range(n_relu_layers):
    W = nn_params[f'W{i}']
    b = nn_params[f'b{i}']
    
    # Calcular bounds de pré-ativação (Wx+b)
    pre_act_bounds = calculate_layer_bounds(current_bounds, W, b)
    all_bounds[f'L{i+1}_pre'] = pre_act_bounds
    
    # Calcular bounds de pós-ativação (ReLU)
    # L_post = max(0, L_pre), U_post = max(0, U_pre)
    post_act_bounds = [(max(0, L), max(0, U)) for L, U in pre_act_bounds]
    all_bounds[f'L{i+1}_post'] = post_act_bounds
    
    # A saída desta camada é a entrada da próxima
    current_bounds = post_act_bounds
    
    print(f"Bounds calculados para Camada ReLU {i+1}.")

# 3. Bounds da Camada Final (Linear)
# A camada final não tem ReLU
W_final = nn_params[f'W{n_relu_layers}']
b_final = nn_params[f'b{n_relu_layers}']
final_bounds = calculate_layer_bounds(current_bounds, W_final, b_final)
all_bounds['Final_Output_Scaled'] = final_bounds

print(f"Bounds Finais (Custo Escalado): [{final_bounds[0][0]:.3f}, {final_bounds[0][1]:.3f}]\n")

# --- INÍCIO DA MODIFICAÇÃO (ADIÇÃO) ---
# Precisamos dos bounds REAIS (des-escalados) do custo para a linearização
y_min_scaled, y_max_scaled = final_bounds[0]

# C_L = Custo Mínimo Real (pode ser negativo se a NN extrapolar mal)
# C_U = Custo Máximo Real
C_L_real = (y_min_scaled / y_scaler.scale_[0]) + y_scaler.min_[0]
C_U_real = (y_max_scaled / y_scaler.scale_[0]) + y_scaler.min_[0]

print(f"Bounds REAIS do Custo (C_L, C_U): [{C_L_real:.2f}, {C_U_real:.2f}]")
# --- FIM DA MODIFICAÇÃO ---

--- Calculando Bounds (Big-M) para todos os neurônios ---
Bounds do Input Escalado da Rede (L0): [0.001, 1.178]
Bounds calculados para Camada ReLU 1.
Bounds calculados para Camada ReLU 2.
Bounds Finais (Custo Escalado): [-0.112, 1.284]

Bounds REAIS do Custo (C_L, C_U): [-2023.96, 23122.94]


In [6]:
# --- Bloco 5: Carregar Dados do Problema (Gurobi) ---
# IDÊNTICO aos notebooks anteriores.

# 48 períodos de 30 minutos
T = range(1, 49)

# Demanda por período (MW)
D_lista = [
    90, 98, 95, 93, 90, 90, 92, 95, 100, 105, 110, 120, 130, 140, 150, 165, 
    180, 195, 210, 225, 240, 245, 250, 248, 245, 240, 230, 225, 220, 215, 
    210, 205, 200, 195, 190, 200, 210, 220, 230, 240, 235, 230, 215, 200, 
    175, 150, 130, 115
]
D = {t: D_lista[t-1] for t in T}

# --- Usina a Gás (Linear) ---
gas_params = {
    'var_cost': 80,    # R$/MW
    'fix_cost': 250,   # R$/período
    'max_cap': 150,    # MW
    'min_gen': 45,     # MW
    'ramp_rate': 50,   # MW/30min
    'min_uptime': 3    # períodos
}

# --- Usina a Carvão (Não-Linear) ---
carvao_params = {
    # Custo Variável: 60*P + 0.2*P^2
    'fix_cost': 1000,         # R$/período
    'max_cap': 200,           # MW
    'min_gen': 30,            # MW
    'ramp_rate': 40,          # MW/30min
    'min_uptime': 1           # períodos
}

In [7]:
# --- Bloco 6: Criação do Modelo Gurobi (MILP-ReLU) ---

m = gp.Model("MILP_Despacho_ReLU")

# Desativar o Presolve (como solicitado)
m.setParam('Presolve', 0)

Set parameter Username
Set parameter LicenseID to value 2638716
Academic license - for non-commercial use only - expires 2026-03-19
Set parameter Presolve to value 0


In [8]:
# --- 7. Definição das Variáveis de Decisão ---

# --- Variáveis Originais ---
T_with_0 = range(0, 49) 
P_gas = m.addVars(T_with_0, name="P_gas", lb=0.0)
P_carvao = m.addVars(T_with_0, name="P_carvao", lb=0.0)
z_gas = m.addVars(T_with_0, name="z_gas", vtype=GRB.BINARY)
z_carvao = m.addVars(T_with_0, name="z_carvao", vtype=GRB.BINARY)
start_gas = m.addVars(T, name="start_gas", vtype=GRB.BINARY)
start_carvao = m.addVars(T, name="start_carvao", vtype=GRB.BINARY)

# --- Novas Variáveis para a Rede Neural (para cada período t) ---

# --- INÍCIO DA MODIFICAÇÃO ---

# C_var_carvao_hat: O custo variável REAL (desnormalizado) que vai para o OBJETIVO.
# Note o lb=0.0 (custo variável não pode ser negativo)
C_var_carvao_hat = m.addVars(
    T, name="C_var_carvao_hat", lb=0.0, ub=max(0, C_U_real)
)

# C_nn_descaled: Variável INTERMEDIÁRIA que recebe a saída da NN.
# Ela pode ser negativa se a NN extrapolar mal.
C_nn_descaled = m.addVars(
    T, name="C_nn_descaled", lb=C_L_real, ub=C_U_real
)

# --- FIM DA MODIFICAÇÃO ---


# Variáveis internas da rede (por período t, por camada l, por neurônio j)

# --- CORREÇÃO AQUI ---
# A variável correta é 'input_U' (com underscore)
nn_input_scaled = m.addVars(T, name="nn_in_scaled", lb=input_L, ub=input_U)
# --- FIM DA CORREÇÃO ---

nn_layer_output = {} # Saída PÓS-ReLU
nn_neuron_binary = {} # Variável 'z' do Big-M

# Criar variáveis para as camadas ReLU
for t in T:
    for l in range(n_relu_layers): # Camadas 0 e 1 (que são L1 e L2)
        n_neurons = nn_params[f'W{l}'].shape[0]
        # Pegar os bounds pós-ativação
        post_bounds = all_bounds[f'L{l+1}_post']
        
        for j in range(n_neurons):
            L, U = post_bounds[j]
            nn_layer_output[t, l, j] = m.addVar(
                name=f"y_l{l}_n{j}_t{t}", lb=L, ub=U
            )
            nn_neuron_binary[t, l, j] = m.addVar(
                name=f"z_l{l}_n{j}_t{t}", vtype=GRB.BINARY
            )

print("Variáveis do Gurobi (incluindo as da rede) criadas.")

Variáveis do Gurobi (incluindo as da rede) criadas.


In [9]:
# --- 8. Definição da Função Objetivo (LINEAR) ---

custo_gas = gp.quicksum(
    gas_params['var_cost'] * P_gas[t] + gas_params['fix_cost'] * z_gas[t]
    for t in T
)

# Substituímos a fórmula quadrática pela nossa variável de custo da rede
custo_carvao_relu = gp.quicksum(
    C_var_carvao_hat[t] + carvao_params['fix_cost'] * z_carvao[t]
    for t in T
)

m.setObjective(custo_gas + custo_carvao_relu, GRB.MINIMIZE)

In [10]:
# --- 9. Definição das Restrições ---

print("Adicionando restrições...")

# --- Restrições Originais (Copiadas) ---
# Estado Inicial (t=0)
m.addConstr(P_gas[0] == 0)
m.addConstr(z_gas[0] == 0)
m.addConstr(P_carvao[0] == 0)
m.addConstr(z_carvao[0] == 0)

# Restrições para cada período t = 1 a 48
for t in T:
    # 1. Atender Demanda
    m.addConstr(P_gas[t] + P_carvao[t] >= D[t], name=f"Demanda_t{t}")
    # 2. Gás
    m.addConstr(P_gas[t] <= gas_params['max_cap'] * z_gas[t], name=f"Gas_MaxCap_t{t}")
    m.addConstr(P_gas[t] >= gas_params['min_gen'] * z_gas[t], name=f"Gas_MinGen_t{t}")
    # 3. Carvão
    m.addConstr(P_carvao[t] <= carvao_params['max_cap'] * z_carvao[t], name=f"Carvao_MaxCap_t{t}")
    m.addConstr(P_carvao[t] >= carvao_params['min_gen'] * z_carvao[t], name=f"Carvao_MinGen_t{t}")
    # 4. Rampas
    m.addConstr(P_gas[t] - P_gas[t-1] <= gas_params['ramp_rate'], name=f"Gas_RampUp_t{t}")
    m.addConstr(P_gas[t-1] - P_gas[t] <= gas_params['ramp_rate'], name=f"Gas_RampDown_t{t}")
    m.addConstr(P_carvao[t] - P_carvao[t-1] <= carvao_params['ramp_rate'], name=f"Carvao_RampUp_t{t}")
    m.addConstr(P_carvao[t-1] - P_carvao[t] <= carvao_params['ramp_rate'], name=f"Carvao_RampDown_t{t}")
    # 5. Partida
    m.addConstr(start_gas[t] >= z_gas[t] - z_gas[t-1], name=f"Gas_StartDef_t{t}")
    m.addConstr(start_carvao[t] >= z_carvao[t] - z_carvao[t-1], name=f"Carvao_StartDef_t{t}")
    # 6. Min Uptime
    k_gas = gas_params['min_uptime']
    for tau in range(t, min(t + k_gas, 49)): m.addConstr(z_gas[tau] >= start_gas[t])
    k_carvao = carvao_params['min_uptime']
    for tau in range(t, min(t + k_carvao, 49)): m.addConstr(z_carvao[tau] >= start_carvao[t])

    # --- NOVAS Restrições da Rede Neural (para cada t) ---
    
    # 7. Ligar P_carvao,t (Real) com o Input Escalado da Rede
    #    x_scaled = (x_real - x_min) * x_scale
    m.addConstr(
        nn_input_scaled[t] == (P_carvao[t] - x_scaler.min_[0]) * x_scaler.scale_[0],
        name=f"NN_Input_Scale_t{t}"
    )
    
    # 8. Restrições "Big-M" para as camadas ReLU
    current_layer_input_vars = [nn_input_scaled[t]] # Começa com o input
    
    for l in range(n_relu_layers): # Camadas 0 e 1 (que são L1 e L2)
        W = nn_params[f'W{l}']
        b = nn_params[f'b{l}']
        n_neurons_out = W.shape[0]
        n_neurons_in = W.shape[1]
        
        # Pegar os bounds pré-calculados
        pre_act_bounds = all_bounds[f'L{l+1}_pre']
        
        # Coletar as variáveis de saída desta camada
        output_vars_list = [nn_layer_output[t, l, j] for j in range(n_neurons_out)]
        
        for j in range(n_neurons_out): # Para cada neurônio na camada
            
            # 1. Definir a pré-ativação (Wx+b)
            pre_act = gp.quicksum(
                W[j, k] * current_layer_input_vars[k] for k in range(n_neurons_in)
            ) + b[j]
            
            # 2. Pegar as variáveis
            y_out = nn_layer_output[t, l, j]
            z_bin = nn_neuron_binary[t, l, j]
            
            # 3. Pegar os bounds L e U (Big-M)
            L, U = pre_act_bounds[j]
            
            # 4. Adicionar as restrições Big-M
            # y_out >= Wx+b
            m.addConstr(y_out >= pre_act, name=f"BigM_1_l{l}_n{j}_t{t}")
            # y_out <= (Wx+b) - L*(1-z)
            m.addConstr(y_out <= pre_act - L * (1 - z_bin), name=f"BigM_2_l{l}_n{j}_t{t}")
            # y_out <= U*z
            m.addConstr(y_out <= U * z_bin, name=f"BigM_3_l{l}_n{j}_t{t}")
            # y_out >= 0 (Já garantido pelo lb=0 da var, mas bom ser explícito)
            # m.addConstr(y_out >= 0, name=f"BigM_4_l{l}_n{j}_t{t}")
            
        # O input da próxima camada é a saída desta
        current_layer_input_vars = output_vars_list
    
    # 9. Ligar a Camada Final (Linear) à Saída Escalada
    # (current_layer_input_vars agora contém as saídas da última camada ReLU)
    W_final = nn_params[f'W{n_relu_layers}']
    b_final = nn_params[f'b{n_relu_layers}']
    
    scaled_output_cost = gp.quicksum(
        W_final[0, k] * current_layer_input_vars[k] for k in range(ARCH_N_NEURONS)
    ) + b_final[0]
    
    # --- INÍCIO DA MODIFICAÇÃO (LÓGICA INTELIGENTE) ---
    
    # 10. Des-normalizar a Saída para a variável INTERMEDIÁRIA
    # C_real = C_scaled / C_scale + C_min
    m.addConstr(
        C_nn_descaled[t] == (scaled_output_cost / y_scaler.scale_[0]) + y_scaler.min_[0],
        name=f"NN_Output_Scale_t{t}"
    )

    # 11. Lógica Inteligente (Linearização de Fortet)
    # Substitui a antiga restrição "Cost_z_link"
    #
    # Modela: C_var_carvao_hat[t] = z_carvao[t] * C_nn_descaled[t]
    #
    z = z_carvao[t]
    C_var = C_var_carvao_hat[t] # Var no objetivo (lb=0)
    C_nn = C_nn_descaled[t]      # Var da NN (pode ser negativa)
    
    # Bounds Reais (calculados no Bloco 4)
    L = C_L_real 
    U = C_U_real 

    # Se z=0 => C_var = 0.
    # Se z=1 => C_var = C_nn.
    m.addConstr(C_var <= U * z, name=f"Fortet_1_t{t}")
    m.addConstr(C_var >= L * z, name=f"Fortet_2_t{t}")
    m.addConstr(C_var <= C_nn - L * (1 - z), name=f"Fortet_3_t{t}")
    m.addConstr(C_var >= C_nn - U * (1 - z), name=f"Fortet_4_t{t}")
    
    # --- FIM DA MODIFICAÇÃO ---
    

print("Todas as restrições (incluindo Big-M) foram adicionadas.")

Adicionando restrições...
Todas as restrições (incluindo Big-M) foram adicionadas.


In [11]:
# --- 10. Otimização do Modelo ---
print("\n--- Iniciando Otimização do Modelo MILP-ReLU ---")
start_time = time.time()

m.optimize()
end_time = time.time()
print("--- Otimização Concluída ---")

solve_time = end_time - start_time


--- Iniciando Otimização do Modelo MILP-ReLU ---
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
Presolve  0



Optimize a model with 14833 rows, 9652 columns and 253294 nonzeros
Model fingerprint: 0x100861d2
Variable types: 4850 continuous, 4802 integer (4802 binary)
Coefficient statistics:
  Matrix range     [4e-05, 2e+04]
  Objective range  [1e+00, 1e+03]
  Bounds range     [1e-03, 2e+04]
  RHS range        [1e-05, 2e+04]
Variable types: 4848 continuous, 4804 integer (4802 binary)
Found heuristic solution: objective 871268.09137

Root relaxation: objective 5.397714e+05, 2905 iterations, 0.06 seconds (0.07 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 539771.381    0  719 871268.091 539771.381  38.0%     -    1s
     0     0 571720.211    0  560 871268.091 571720.211  34.4%     -    1s
H    0     0                    756914.93837 571720.211  24.5%     -    2s
     0     0 676471.019    0  668 756914.938 676471.019  10.6%     -    3s
     0     0 676746.816    0  66

In [12]:
# --- 11. Exibição dos Resultados ---
if m.Status == GRB.OPTIMAL:
    print(f"\nSolução Ótima Encontrada!")
    print(f"Custo Total (MILP-ReLU): R$ {m.ObjVal:.2f}")
    print(f"Tempo de Solução: {solve_time:.4f} segundos")
    print("-" * 70)
    
    print("Resumo da Operação (Geração em MW):")
    print("Per. | Demanda | Gás (P)   | Gás (z) | Carvão (P) | Carvão (z) | Total Gen. | Sobra")
    print("-" * 70)
    for t in T:
        gas_gen = P_gas[t].X
        gas_z = "ON" if z_gas[t].X > 0.5 else "OFF"
        carvao_gen = P_carvao[t].X
        carvao_z = "ON" if z_carvao[t].X > 0.5 else "OFF"
        total_gen = gas_gen + carvao_gen
        sobra = total_gen - D[t]
        
        print(f" {t:2d} |  {D[t]:3d}  | {gas_gen:8.2f} | {gas_z:^7} | {carvao_gen:10.2f} | {carvao_z:^10} | {total_gen:10.2f} | {sobra:5.2f}")

elif m.Status == GRB.INFEASIBLE:
    print("\n--- ERRO: Modelo Inviável ---")
    m.computeIIS()
    m.write("modelo_conflito.ilp")
    print("IIS salvo em 'modelo_conflito.ilp'.")
else:
    print(f"Otimização terminou com status: {m.Status}")


Solução Ótima Encontrada!
Custo Total (MILP-ReLU): R$ 735363.86
Tempo de Solução: 16.6902 segundos
----------------------------------------------------------------------
Resumo da Operação (Geração em MW):
Per. | Demanda | Gás (P)   | Gás (z) | Carvão (P) | Carvão (z) | Total Gen. | Sobra
----------------------------------------------------------------------
  1 |   90  |    50.00 |   ON    |      40.00 |     ON     |      90.00 |  0.00
  2 |   98  |    98.00 |   ON    |       0.00 |    OFF     |      98.00 |  0.00
  3 |   95  |    95.00 |   ON    |       0.00 |    OFF     |      95.00 |  0.00
  4 |   93  |    93.00 |   ON    |       0.00 |    OFF     |      93.00 |  0.00
  5 |   90  |    90.00 |   ON    |       0.00 |    OFF     |      90.00 |  0.00
  6 |   90  |    90.00 |   ON    |       0.00 |    OFF     |      90.00 |  0.00
  7 |   92  |    92.00 |   ON    |       0.00 |    OFF     |      92.00 |  0.00
  8 |   95  |    95.00 |   ON    |       0.00 |    OFF     |      95.00 |  0.0